In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# wd = '/mnt/openfact/users/msawinski/factue-task2'
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [3]:
import pandas as pd
from pathlib import Path
import os
from factue.utils.viz import disp
import numpy as np

# judge


In [4]:

root = Path("data/llm_output/normalization/judge")
# Example path template
files = root.rglob("LLAMA_31_8B/**/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]
# df['init_part_no'] = df.source_file.astype(str).str.split('/',expand=True)[9].str.split('_',expand=True)[1].str.split('.',expand=True)[0].astype(int)
df['lang'] = df.source_file.astype(str).str.split('/',expand=True)[8]
df['split'].value_counts()

split
dev    10
Name: count, dtype: int64

In [5]:
df

,claim_candidate,claim_improved,text,reference,split,lang,init_part_no,original_index,output_id,prompt_name,...,model_name,model_mode,raw,think_content,is_valid,status,plain_content,claim,error,source_file
0,"[A Sikh man with no mustache is present at the farmers' protest in Delhi, which, according to astronomical data, occurs approximately once per century., A Sikh man without a mustache appears at th...","[A Sikh with no mustache appeared at the Delhi farmers' protest, which is reportedly an astronomical occurrence happening approximately once every 100 years., A Sikh appeared at the Delhi farmers ...",Sikh with no mustache in Delhi farmers protest. According to astronomy such an event happens once in a century,'Sardar with no mustache' involved in farmers protest,dev,dev-eng,43,0,,default,...,llama3.1:8b,chat,"[A Sikh appeared at the Delhi farmers protest without a mustache, which according to astronomy is a rare event that occurs once in a century.]",[None],[True],[no validation],"[A Sikh appeared at the Delhi farmers protest without a mustache, which according to astronomy is a rare event that occurs once in a century.]","A Sikh appeared at the Delhi farmers protest without a mustache, which according to astronomy is a rare event that occurs once in a century.",None,data/llm_output/normalization/judge/LLAMA_31_8B/v001/default/dev/dev-eng/dev-eng_part_0000.parquet
1,"[Challenging people to wear face coverings may result in a fine of up to £5,000 and compensation payments for disability discrimination., Challenging people to wear face coverings may result in pe...","[Challenging people to wear face coverings may result in a fine of up to £5,000 and compensation payments for disability discrimination., Challenging people to wear face coverings may result in pe...","The government advice is not to challenge people to wear a face covering. This is for GOOD REASON. If you do so, you and your employees may be PERSONALLY LIABLE for AN OFFENCE liable on a summary ...",The government advice is not to challenge someone who is not wearing a face covering.,dev,dev-eng,31,1,,default,...,llama3.1:8b,chat,"[Challenging people to wear face coverings may result in a fine of up to £5,000 and compensation payments for disability discrimination.]",[None],[True],[no validation],"[Challenging people to wear face coverings may result in a fine of up to £5,000 and compensation payments for disability discrimination.]","Challenging people to wear face coverings may result in a fine of up to £5,000 and compensation payments for disability discrimination.",None,data/llm_output/normalization/judge/LLAMA_31_8B/v001/default/dev/dev-eng/dev-eng_part_0000.parquet
2,"[The Bharatiya Janata Party (BJP) government will return in 2024., The government of Bharatiya Janata Party (BJP) will return in 2024., The BJP government will return in 2024.]","[Will the Bharatiya Janata Party (BJP) government return in 2024!!!, BJP government will return in 2024!!!, The BJP government will return in 2024.]",Will our BJP\ngovernment return in 2024!!!,Will our BJP government return in 2024,dev,dev-eng,34,2,,default,...,llama3.1:8b,chat,[The BJP government will return in 2024!!!],[None],[True],[no validation],[The BJP government will return in 2024!!!],The BJP government will return in 2024!!!,None,data/llm_output/normalization/judge/LLAMA_31_8B/v001/default/dev/dev-eng/dev-eng_part_0000.parquet
3,"[Tripti Desai converted to Christianity three years ago and is allegedly closely connected to the conversion racket., Truth out: Tripti Desai converted to Christianity three years back., Truth out...","[Tripti Desai converted to Christianity three years ago and is closely connected to a conversion racket., Tripti Desai converted to Christianity three years back, specifically in 2013., Tripti Des...",Ok truth out #TriptiDesai converted to christianity three years back..in 2013..closely connected to conversion racket...she is a CHRISTIAN in hindu garb...beware keralites..come to dest

In [18]:
preview_cols = ['prompt_version','model_name','text','reference', 'claim','claim_improved','claim_candidate','think_content']
sample  = df[preview_cols].sort_values(['text',"model_name", 'prompt_version'])
sample

,prompt_version,model_name,text,reference,claim,claim_improved,claim_candidate,think_content
6,v001,llama3.1:8b,"Just to let you know that the first thing that’ll strike you when you look up this act on the government website is that it says changes to legislation, and this has been updated on the 27th April...",The Control of Diseases Act in the UK changed on 27 April.,"Changes to legislation were updated on the government website as of the 27th April 2020, and have since come into force.","[Changes to legislation were updated on the government website as of the 27th April 2020, and have since come into force., Changes to legislation came into effect on the 27th of April, 2020., Chan...","[Changes to legislation have come into effect as of April 27, 2020., The Act regarding changes to legislation came into force on April 27, 2020., The Changes to Legislation have taken effect as of...",[None]
4,v001,llama3.1:8b,"MUGABE'S CASEKET\n17billion worth of Robert Mugabe's casket\nhas been computerized in such a way that after his interment, his family can watch his remains through their phone devices...\n\nAccord...",Mugabe has computerised coffin,"Robert Mugabe's casket, worth 17 billion, has been computerized to protect his remains from decomposing for ten years and allow his family to watch them via phone devices after interment.","[Robert Mugabe's casket has been computerized with facilities that protect his remains from decomposing for the next ten years, allowing his family to view them remotely via phone devices., Robert...","[Robert Mugabe's casket has been computerized to allow his family to watch his remains through their phone devices for up to ten years after interment., Robert Mugabe's casket, worth 17 billion, h...",[None]
3,v001,llama3.1:8b,Ok truth out #TriptiDesai converted to christianity three years back..in 2013..closely connected to conversion racket...she is a CHRISTIAN in hindu garb...beware keralites..come to destroy #Sabari...,Trupti Desai Converted To Christianity,"Tripti Desai converted to Christianity three years back, specifically in 2013 and is closely connected to a conversion racket.","[Tripti Desai converted to Christianity three years ago and is closely connected to a conversion racket., Tripti Desai converted to Christianity three years back, specifically in 2013., Tripti Des...","[Tripti Desai converted to Christianity three years ago and is allegedly closely connected to the conversion racket., Truth out: Tripti Desai converted to Christianity three years back., Truth out...",[None]
0,v001,llama3.1:8b,Sikh with no mustache in Delhi farmers protest. According to astronomy such an event happens once in a century,'Sardar with no mustache' involved in farmers protest,"A Sikh appeared at the Delhi farmers protest without a mustache, which according to astronomy is a rare event that occurs once in a century.","[A Sikh with no mustache appeared at the Delhi farmers' protest, which is reportedly an astronomical occurrence happening approximately once every 100 years., A Sikh appeared at the Delhi farmers ...","[A Sikh man with no mustache is present at the farmers' protest in Delhi, which, according to astronomical data, occurs approximately once per century., A Sikh man without a mustache appears at th...",[None]
5,v001,llama3.1:8b,"Steve Kirsch, the Executive Director of the COVID-19 Early Treatment Fund was up next to testify before the FDA. Kirsch argued the vaccines kill more people than they save.Steve Kirsch also point...",Covid-19 vaccines kill more people then they save,"COVID-19 vaccines kill more people than they save, according to Kirsch's argument before the FDA.","[COVID-19 vaccines kill more people than they save, according to Steve Kirsch's argument during an FDA hearing., COVID-19 vaccines kill more people than they save, according to Kirsch's argument b...","[Government data demonstrates that COVID shots may kill more than they save., Government data demonstrates that COVID shots may kil

In [7]:
disp(sample)

,prompt_version,model_name,text,reference,claim,claim_improved,claim_candidate,think_content
6,v001,llama3.1:8b,"Just to let you know that the first thing that’ll strike you when you look up this act on the government website is that it says changes to legislation, and this has been updated on the 27th April 2020. So just a day or so ago. And it’s come into force on that day. So this is applicable from now.",The Control of Diseases Act in the UK changed on 27 April.,"Changes to legislation were updated on the government website as of the 27th April 2020, and have since come into force.","['Changes to legislation were updated on the government website as of the 27th April 2020, and have since come into force.' 'Changes to legislation came into effect on the 27th of April, 2020.' 'Changes to legislation have been updated and are in effect since April 27th, 2020.']","['Changes to legislation have come into effect as of April 27, 2020.' 'The Act regarding changes to legislation came into force on April 27, 2020.' 'The Changes to Legislation have taken effect as of April 27th, 2020.']",[None]
4,v001,llama3.1:8b,"MUGABE'S CASEKET 17billion worth of Robert Mugabe's casket has been computerized in such a way that after his interment, his family can watch his remains through their phone devices... According to the London Casket makers that supplied the casket,the facilities in it will protect his remains from decomposing for the next ten years...this however implies that,apart from relating with his family, any member of his family that wishes to watch his remains in the tomb will have to access same via his or her phone devices. The white is still at the top using our brains MUGABE'S CASEKET 17billion worth of Robert Mugabe's casket has been computerized in such a way that after his interment, his family can watch his remains through their phone devices... According to the London Casket makers that supplied the casket,the facilities in it will protect his remains from decomposing for the next ten years...this however implies that,apart from relating with his family, any member of his family that wishes to watch his remains in the tomb will have to access same via his or her phone devices. The white is still at the top using our brains MUGABE'S CASEKET 17billion worth of Robert Mugabe's casket has been computerized in such a way that after his interment, his family can watch his remains through their phone devices... According to the London Casket makers that supplied the casket,the facilities in it will protect his remains from decomposing for the next ten years...this however implies that,apart from relating with his family, any member of his family that wishes to watch his remains in the tomb will have to access same via his or her phone devices. The white is still at the top using our brains None",Mugabe has computerised coffin,"Robert Mugabe's casket, worth 17 billion, has been computerized to protect his remains from decomposing for ten years and allow his family to watch them via phone devices after interment.","[""Robert Mugabe's casket has been computerized with facilities that protect his remains from decomposing for the next ten years, allowing his family to view them remotely via phone devices."" ""Robert Mugabe's casket, worth 17 billion, has been computerized to protect his remains from decomposing for ten years and allow his family to watch them via phone devices after interment."" ""Robert Mugabe's casket has been computerized to protect his remains for ten years and allow his family to watch them through their phone devices.""]","[""Robert Mugabe's casket has been computerized to allow his family to watch his remains through their phone devices for up to ten years after interment."" ""Robert Mugabe's casket, worth 17 billion, has been computerized so that his family can watch his remains through their phone devices after interment."" ""Robert Mugabe's casket has been computerized to protect his remains for ten years an

In [21]:
import nltk
# nltk.download('punkt', force=True)
nltk.download("punkt_tab")
nltk.download('wordnet')
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/marcinsawinski/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/marcinsawinski/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
def add_meteor_score(df, ref_col='reference', cand_col='claim'):
    def compute_score(row):
        ref = row[ref_col]
        cand = row[cand_col]
        if pd.notnull(ref) and pd.notnull(cand):
            return meteor_score(
                [word_tokenize(ref)],
                word_tokenize(cand))
        return None  # or 0.0
    df['meteor_score'] = df.apply(compute_score, axis=1).round(3)
    return df
df = add_meteor_score(df, ref_col='reference', cand_col='claim')


In [26]:
prev = df[preview_cols +['meteor_score']]
prev

,prompt_version,model_name,text,reference,claim,claim_improved,claim_candidate,think_content,meteor_score
0,v001,llama3.1:8b,Sikh with no mustache in Delhi farmers protest. According to astronomy such an event happens once in a century,'Sardar with no mustache' involved in farmers protest,"A Sikh appeared at the Delhi farmers protest without a mustache, which according to astronomy is a rare event that occurs once in a century.","[A Sikh with no mustache appeared at the Delhi farmers' protest, which is reportedly an astronomical occurrence happening approximately once every 100 years., A Sikh appeared at the Delhi farmers ...","[A Sikh man with no mustache is present at the farmers' protest in Delhi, which, according to astronomical data, occurs approximately once per century., A Sikh man without a mustache appears at th...",[None],0.292
1,v001,llama3.1:8b,"The government advice is not to challenge people to wear a face covering. This is for GOOD REASON. If you do so, you and your employees may be PERSONALLY LIABLE for AN OFFENCE liable on a summary ...",The government advice is not to challenge someone who is not wearing a face covering.,"Challenging people to wear face coverings may result in a fine of up to £5,000 and compensation payments for disability discrimination.","[Challenging people to wear face coverings may result in a fine of up to £5,000 and compensation payments for disability discrimination., Challenging people to wear face coverings may result in pe...","[Challenging people to wear face coverings may result in a fine of up to £5,000 and compensation payments for disability discrimination., Challenging people to wear face coverings may result in pe...",[None],0.289
2,v001,llama3.1:8b,Will our BJP\ngovernment return in 2024!!!,Will our BJP government return in 2024,The BJP government will return in 2024!!!,"[Will the Bharatiya Janata Party (BJP) government return in 2024!!!, BJP government will return in 2024!!!, The BJP government will return in 2024.]","[The Bharatiya Janata Party (BJP) government will return in 2024., The government of Bharatiya Janata Party (BJP) will return in 2024., The BJP government will return in 2024.]",[None],0.771
3,v001,llama3.1:8b,Ok truth out #TriptiDesai converted to christianity three years back..in 2013..closely connected to conversion racket...she is a CHRISTIAN in hindu garb...beware keralites..come to destroy #Sabari...,Trupti Desai Converted To Christianity,"Tripti Desai converted to Christianity three years back, specifically in 2013 and is closely connected to a conversion racket.","[Tripti Desai converted to Christianity three years ago and is closely connected to a conversion racket., Tripti Desai converted to Christianity three years back, specifically in 2013., Tripti Des...","[Tripti Desai converted to Christianity three years ago and is allegedly closely connected to the conversion racket., Truth out: Tripti Desai converted to Christianity three years back., Truth out...",[None],0.478
4,v001,llama3.1:8b,"MUGABE'S CASEKET\n17billion worth of Robert Mugabe's casket\nhas been computerized in such a way that after his interment, his family can watch his remains through their phone devices...\n\nAccord...",Mugabe has computerised coffin,"Robert Mugabe's casket, worth 17 billion, has been computerized to protect his remains from decomposing for ten years and allow his family to watch them via phone devices after interment.","[Robert Mugabe's casket has been computerized with facilities that protect his remains from decomposing for the next ten years, allowing his family to view them remotely via phone devices., Robert...","[Robert Mugabe's casket has been computerized to allow his family to watch his remains through their phone devices for up to ten years after interment., Robert Mugabe's casket, worth 17 billion, h...",[None],0.214
5,v001,llama3.1:8b,"Steve Kirsch, the Executive Director of the COVID-19 Early Treatment Fund was up next to testify before the FDA. Kirsch

In [27]:
disp(prev)

,prompt_version,model_name,text,reference,claim,claim_improved,claim_candidate,think_content,meteor_score
0,v001,llama3.1:8b,Sikh with no mustache in Delhi farmers protest. According to astronomy such an event happens once in a century,'Sardar with no mustache' involved in farmers protest,"A Sikh appeared at the Delhi farmers protest without a mustache, which according to astronomy is a rare event that occurs once in a century.","[""A Sikh with no mustache appeared at the Delhi farmers' protest, which is reportedly an astronomical occurrence happening approximately once every 100 years."" 'A Sikh appeared at the Delhi farmers protest without a mustache, which according to astronomy is a rare event that occurs once in a century.' ""A rare astronomical event is occurring at the Delhi farmers' protest, where a Sikh man with no mustache appears once in every 100 years.""]","[""A Sikh man with no mustache is present at the farmers' protest in Delhi, which, according to astronomical data, occurs approximately once per century."" 'A Sikh man without a mustache appears at the Delhi farmers protest. This is an astronomical occurrence, happening only once in 100 years.' ""A man without a mustache appears at the Delhi farmers' protest.""]",[None],0.292000
1,v001,llama3.1:8b,"The government advice is not to challenge people to wear a face covering. This is for GOOD REASON. If you do so, you and your employees may be PERSONALLY LIABLE for AN OFFENCE liable on a summary conviction to pay a fine of up £5,000 - section 112 (Aiding contraventions) of the Equality Act 2010. “AN ACT OF DISABILITY DISCRIMINATION and be ordered to pay any individual who suffers injury to feelings compensation between £900 and £9,000 - section 119 (Remedies) of the Equality Act 2010 [sic].",The government advice is not to challenge someone who is not wearing a face covering.,"Challenging people to wear face coverings may result in a fine of up to £5,000 and compensation payments for disability discrimination.","['Challenging people to wear face coverings may result in a fine of up to £5,000 and compensation payments for disability discrimination.' 'Challenging people to wear face coverings may result in personal liability for a fine up to £5,000 or compensation payments.' 'Wearing a face covering may result in personal liability for an offence punishable by up to a fine of £5,000 under section 112 of the Equality Act 2010.']","['Challenging people to wear face coverings may result in a fine of up to £5,000 and compensation payments for disability discrimination.' 'Challenging people to wear face coverings may result in personal liability for a fine up to £5,000 or compensation payments.' 'Wearing a face covering may result in personal liability for an offence punishable by up to a fine of £5,000 under section 112 of the Equality Act 2010.']",[None],0.289000
2,v001,llama3.1:8b,Will our BJP government return in 2024!!!,Will our BJP government return in 2024,The BJP government will return in 2024!!!,['Will the Bharatiya Janata Party (BJP) government return in 2024!!!' 'BJP government will return in 2024!!!' 'The BJP government will return in 2024.'],['The Bharatiya Janata Party (BJP) government will return in 2024.' 'The government of Bharatiya Janata Party (BJP) will return in 2024.' 'The BJP government will return in 2024.'],[None],0.771000
3,v001,llama3.1:8b,Ok truth out #TriptiDesai converted to christianity three years back..in 2013..closely connected to conversion racket...she is a CHRISTIAN in hindu garb...beware keralites..come to destroy #Sabarimala #JanamTV report exposes this..,Trupti Desai Converted To Christianity,"Tripti Desai converted to Christianity three years back, specifically in 2013 and is closely connected to a conversion racket.","['Tripti Desai converted to Christianity three years ago and is closely connected to a conversion racket.' 'Tripti Desai converted to Christianity three years back, specifically in 2013.' 'Tripti Desai converted to Ch